In [13]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from matplotlib.colors import hsv_to_rgb
from numpy.fft import fft2, ifft2, fftshift, ifftshift

from skimage import io
import glob
from PIL import Image
import pickle
import os

import waveorder as wo

%load_ext autoreload
%autoreload 2
%matplotlib inline
plt.style.use(['dark_background']) # Plotting option for dark background


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
#Reading files using optimized method (according to IO Speeds notebook from czbiohub/virusLifecycle).
def optimizedReading(img_paths):
    """This function takes in a list of image paths i.e. ["../path1, ../path2"] 
    and reads each image into a numpy array."""
    
    length = len(img_paths)
    empty = np.empty((length, 2048, 2048), dtype = 'uint16')
    
    for num, img in zip(np.arange(length), img_paths):
        empty[num,:,:] = io.imread(img)
    
    return empty

In [2]:
data_path     = '/mnt/comp_micro/SpinningDisk/RawData/virtualstaining/2019_04_08_U2OS_plin2/2019_04_08_U2OS_plin2_Lipid_MT_Actin_3/2-Pos_001_002/'        # data path
bg_path       = '/mnt/comp_micro/SpinningDisk/RawData/virtualstaining/2019_04_08_U2OS_plin2/BG_2019_0408_1412_1/Pos0/'


In [6]:
filedir = data_path + '*PolState*.tif'
filedir_bg = bg_path + '*.tif'
files = sorted(glob.glob(filedir), key=wo.numericalSort)
files_bg = sorted(glob.glob(filedir_bg), key=wo.numericalSort)

In [7]:
files[0]

'/mnt/comp_micro/SpinningDisk/RawData/virtualstaining/2019_04_08_U2OS_plin2/2019_04_08_U2OS_plin2_Lipid_MT_Actin_3/2-Pos_001_002/img_000000000_Zyla_PolState0_Widefield_000.tif'

In [8]:
#select a FOV for processing

Ns = 1000
Ms = 1000
n_start = [250, 250] # grating target

In [9]:
%%timeit
img = plt.imread(files[0]).astype('float64')[n_start[0]:n_start[0]+Ns, n_start[1]:n_start[1]+Ms]

30.1 s ± 9.23 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
%%timeit
img = cv2.imread(files[0]).astype('float64')[n_start[0]:n_start[0]+Ns, n_start[1]:n_start[1]+Ms]

128 ms ± 1.17 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [15]:
%%timeit
img = io.imread(files[0]).astype('float64')[n_start[0]:n_start[0]+Ns, n_start[1]:n_start[1]+Ms]

95.1 ms ± 586 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Reading Lists of filenames: Append vs Writing into empty array


In [22]:
%%timeit
imgs = optimizedReading(files[:50])

5.28 s ± 257 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [23]:
%%timeit
imgs = []
for i in np.arange(50):
    img = io.imread(files[i])
    imgs.append(img)
np.array(imgs)

4.4 s ± 80.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
